**🌟📅 Weekly 1.2 📅 🌟**

This note book is building Cal 1.1

- Making the event class accommodate 7 days
- Making the Journe class accommodate 7 days
- Create a payload function for Journe that extracts all event data

[in this doc](https://docs.google.com/document/d/1WLt353a3DoAwphp6nxL84ZSbf1qMAhs5Y54ZfyKMvsA/edit) ---> 1.2 UP UP

In [ ]:
import datetime

"""
An event is somthing that is planned to be done... it can be a BLOCK or a TASK

BLOCK or TASK will be specified through the bool field of is_block.

All events must have an event_name and an event_group (if no group is given the
group should default to "misc").

When an event is being created, user will get get a prompt. This prompt will
change based on if we are populating a BLOCK or a TASK, idea is to mimic UI.

BLOCK PROMPTS -> when does event start/end? ...
TASK PROMPTS -> how long will it take? ....

HOW TIME WORKS.... Each day is divided into 1440 minute slices.. each slice can
have ONLY ONE time event assigned to it... Each day slice is representative
of a minute in the day.

This is cool beans because

00:00 ----> slice position 0
00:10 ----> slice position 10
01:00 ----> slice position 60
"""

class Event:

  def __init__(self,
               is_block:bool,
               event_name:str,
               event_group="misc"):
    self.event_name = event_name
    self.event_group = event_group
    self.is_block = is_block

    if is_block:
      # grab inputs
      self.event_start_time = str(input(f"When does {self.event_name} start HH:MM format: "))
      self.event_end_time   = str(input(f"When does {self.event_name} end HH:MM format: "))
      self.day_of_wk = int(input(f"What day (0-7): "))
      self.timestamp = (datetime.datetime.now() + datetime.timedelta(days=self.day_of_wk)).strftime("%d-%m-%Y")


      self.event_start_pos  = to_min_unit_of_the_day(self.event_start_time)
      self.event_end_pos  = to_min_unit_of_the_day(self.event_end_time)
      # derive event duration
      self.duration = self.event_end_pos - self.event_start_pos

    else:
      self.duration = int(input(f"How long will {self.event_name} take (in min)?: "))

    print(self)

  def __str__(self):
    if not self.is_block:
      return f"{self.event_name} will take {self.duration} minutes"
    return f"{self.event_name} will take place between {self.event_start_time} and {self.event_end_time} and will take {self.duration} minutes on day {self.day_of_wk}"

"""
Event group .... it is just a container that holds events

"""
class Event_group:

  def __init__(self, event_group_name):
    self.event_group_name = event_group_name
    self.event_list = {}


class Journe:

  def __init__(self):
    self.event_groups = {"misc":Event_group("misc")}
    self.event_group_names = ["misc"]
    self.event_id = 0
    self.events = {"Tasks": {}, "Blocks": {}}

  def export_payload(self):
    payload = {'tasks' : {}, 'blocks' : {}}
    _tasks = self.events["Tasks"]
    _blocks =  self.events["Blocks"]
    # creating tasks
    for _id, _event in _tasks.items():
      payload['tasks'][_id] = {'name':_event.event_name,
                               'duration':_event.duration,
                               'group':_event.event_group
                               }
    # creating blocks
    for _id, _event in _blocks.items():
      payload['blocks'][_id] = {'name':_event.event_name,
                               'duration':_event.duration,
                               'group':_event.event_group,
                               'start_time' : _event.event_start_time,
                               'end_time' : _event.event_end_time,
                               'date' : _event.timestamp
                               }
    return payload

  def add_event_group(self, event_group_name:str):
    event_group_name = event_group_name.lower()
    # check if event group has already been created or not
    if event_group_name in self.event_group_names:
      print("This group is already in there!")
    else:
      self.event_groups[event_group_name] = Event_group(event_group_name)
      self.event_group_names.append(event_group_name)

  def add_event(self, e):
    # check if event_group exists
    if e.event_group.lower() not in self.event_group_names:
      print("Event group does not exist, please create first")
      return
    if e.is_block:
      self.events["Blocks"][self.event_id] = e
      self.event_groups[e.event_group.lower()].event_list[self.event_id] = e

    else:
      self.events["Tasks"][self.event_id] = e
      self.event_groups[e.event_group.lower()].event_list[self.event_id] = e
    self.event_id += 1;

    print(f"{e.event_name} was added to your Journe")
    return

  def show_events(self):
    print(f'there are {len(self.events["Blocks"]) + len(self.events["Tasks"])} events to-do')
    print(f'TASKS')
    [print(e) for e in self.events["Tasks"].values()]
    print(f'BLOCKS')
    [print(e) for e in self.events["Blocks"].values()]

  def draw_events(self):
    # expanding for week!
    week_timelines = [[" " * (24 * 60)] for _ in range(7)]  # Initialize timelines for 7 days a day (24 hours * 60 minutes)
    time_markers = ["{:02d}:{:02d}".format(h, m) + " "*25  for h in range(24) for m in (0, 30)]
    day_names = ["Day 0", "Day 1", "Day 2", "Day 3", "Day 4", "Day 5", "Day 6"]

    for event in self.events["Blocks"].values():
      event_name = event.event_name[:event.event_end_pos - event.event_start_pos].ljust(event.event_end_pos - event.event_start_pos, "#")
      for minute in range(event.event_start_pos, event.event_end_pos):
        week_timelines[event.day_of_wk][0] = week_timelines[event.day_of_wk][0][:minute] + event_name[minute - event.event_start_pos] + week_timelines[event.day_of_wk][0][minute + 1:]

    # printing every day one by one
    print("Week Cal:")
    print(6*" " + "".join(time_markers))
    print("-" * len(week_timelines[0][0]))
    for day, timeline in enumerate(week_timelines):
      print(f"Day{day}: " + timeline[0])
      print("-" * len(timeline[0]))


# helper functions

# returns the minute block position of the day given the HH:MM time
def to_min_unit_of_the_day(str_time : str):
  h, m = str_time.split(":")
  return int(h) * 60 + int(m)

In [ ]:
j = Journe()

In [ ]:
# creating events ...
# mock block
block_1 = Event(True, "cat play time")
block_2 = Event(True, "baby bubi")
block_3 = Event(True, "Bike", "Sports")
# mock task
task_1 = Event(is_block=False, event_name="print stuff out")

When does cat play time start HH:MM format: 10:00
When does cat play time end HH:MM format: 11:00
What day (0-7): 2
cat play time will take place between 10:00 and 11:00 and will take 60 minutes on day 2
When does baby bubi start HH:MM format: 04:00
When does baby bubi end HH:MM format: 05:48
What day (0-7): 1
baby bubi will take place between 04:00 and 05:48 and will take 108 minutes on day 1
When does Bike start HH:MM format: 07:02
When does Bike end HH:MM format: 07:09
What day (0-7): 0
Bike will take place between 07:02 and 07:09 and will take 7 minutes on day 0
How long will print stuff out take (in min)?: 45
print stuff out will take 45 minutes


In [ ]:
j.add_event_group("Sports")
j.add_event(block_1)
j.add_event(block_2)
j.add_event(block_3)
j.add_event(task_1)

cat play time was added to your Journe
baby bubi was added to your Journe
Bike was added to your Journe
print stuff out was added to your Journe


In [ ]:
j.draw_events()

Week Cal:
      00:00                         00:30                         01:00                         01:30                         02:00                         02:30                         03:00                         03:30                         04:00                         04:30                         05:00                         05:30                         06:00                         06:30                         07:00                         07:30                         08:00                         08:30                         09:00                         09:30                         10:00                         10:30                         11:00                         11:30                         12:00                         12:30                         13:00                         13:30                         14:00                         14:30                         15:00                         15:30                         16:00                   

In [ ]:
j.export_payload()

{'tasks': {3: {'name': 'print stuff out', 'duration': 45, 'group': 'misc'}},
 'blocks': {0: {'name': 'cat play time',
   'duration': 60,
   'group': 'misc',
   'start_time': '10:00',
   'end_time': '11:00',
   'date': '12-09-2023'},
  1: {'name': 'baby bubi',
   'duration': 108,
   'group': 'misc',
   'start_time': '04:00',
   'end_time': '05:48',
   'date': '11-09-2023'},
  2: {'name': 'Bike',
   'duration': 7,
   'group': 'Sports',
   'start_time': '07:02',
   'end_time': '07:09',
   'date': '10-09-2023'}}}